In [1]:
import numpy as np
import pandas as pd

In [2]:
###
# Lendo arquivos 
###
time_limit = 30
distance_limit = 1

df_original = pd.read_csv(f"to_index_table_macacos_micos_{time_limit}d_{distance_limit}km.csv",index_col=0)
df_padronizado = pd.read_csv(f"to_index_table_macacos_micos_{time_limit}d_{distance_limit}km_padronizado.csv",index_col=0)

In [ ]:
##
# Filtrar os clusters confirmados
##
cluster_fil = df_padronizado[df_original["confirmado"] != 0].sort_values(by="confirmado")

In [6]:

df = cluster_fil.copy() # 'df' é o seu DataFrame filtrado com registros confirmados.                
evaluation_count = 0    # contar avaliações
col = ["a_quant", "num_reg", "intervalo", "freq_num_reg", "freq_a_quant", "extensao", "perc_mortos","freq_morto"]
var = len(col) # número de variáveis

def objective(weights):    
    global evaluation_count
    index_values = (
        weights[0] * df['a_quant'] +
        weights[1] * df['num_reg'] +
        weights[2] * df['intervalo'] +
        weights[3] * df['freq_num_reg'] +
        weights[4] * df['freq_morto'] +
        weights[5] * df['freq_a_quant'] +
        weights[6] * df['perc_mortos'] +
        weights[7] * df['extensao']
    )
    obj1 = - index_values.sum()  # maximizar
    #obj2 = np.std(index_values)  # minimizar
    evaluation_count += 1

    return obj1

def normalize_weights(weights):
    return weights / np.sum(weights)  # Normaliza para que a soma seja 1

# Definindo os limites para os pesos
bounds = [(0, 1) for _ in range(var)]  # Pesos entre 0 e 1
b0 = 0
b1 = 1

def differential_evolution(objective, bounds, population_size=20, F=0.5, CR=0.7, max_generations=100, seed=None):
    if seed is not None:
        np.random.seed(seed)  # Define a semente 

    # Inicializa a população aleatória
    population = np.random.rand(population_size, var)   
    pop_solution = [float('inf')] * population_size

    for i in range(population_size):
        population[i] = normalize_weights(population[i])  # Normaliza a população inicial
        pop_solution[i] = objective(population[i])
    
    for i in range(var):
        population[:, i] = population[:, i] * (bounds[i][1] - bounds[i][0]) + bounds[i][0]

    best_solution = None
    best_score = float('inf')

    for generation in range(max_generations):
        new_population = np.copy(population)
        new_pop_solution = np.copy(pop_solution)

        for i in range(population_size):
            # Seleção de três indivíduos aleatórios diferentes
            indices = list(range(population_size))
            indices.remove(i)
            ind1, ind2, ind3 = np.random.choice(indices, 3, replace=False)

            # Geração de um novo vetor candidato
            v = population[ind1] + F * (population[ind2] - population[ind3])
            v = np.clip(v, [b[0] for b in bounds], [b[1] for b in bounds])  # Limita os valores (ex: np.clip(array, min, max) )
            # v = np.clip(v,b0,b1)

            # Crossover
            trial = np.zeros_like(v)
            for j in range(len(v)):
                if np.random.rand() < CR or j == np.random.randint(len(v)):
                    trial[j] = v[j]
                else:
                    trial[j] = population[i][j]
            '''
            # crossover é um array booleano. 
            # se True (indica que o valor será tomado de mutant) 
            # se False (indica que o valor será mantido de population[i])
            crossover = ( np.random.rand(var) < CR or j == np.random.randint(len(v)) )
            trial = np.where(crossover, v, population[i])
            '''

            # Normaliza o vetor candidato
            trial = normalize_weights(trial)            

            # Avalia o vetor candidato
            trial_score = objective(trial)

            # Substituição
            if trial_score < pop_solution[i]:
                new_population[i] = trial
                new_pop_solution[i] = trial_score

        # Atualiza a população
        population = new_population
        pop_solution = new_pop_solution
        
        # Avalia a melhor solução da população atual
        for i in range(population_size):
            score = pop_solution[i]
            if score < best_score:
                best_solution = population[i]
                best_score = score

        print(f"[{generation}] Best: {best_solution}, Fit: {best_score}, nEval: {evaluation_count}")

    return best_solution, best_score


# Evolução Diferencial
semente = 1
optimal_weights, optimal_value = differential_evolution(objective, bounds, seed=semente)

print("\n Best solution:", optimal_weights)
print("Best solution value:", optimal_value)
#print("Soma dos pesos:", np.sum(optimal_weights)) 

[0] Best: [0.034763   0.078159   0.16398652 0.27144365 0.15706349 0.00522041
 0.22414151 0.06522243], Fit: -42.808659983612515, nEval: 40
[1] Best: [0.05685029 0.02528978 0.05596692 0.25269467 0.13122979 0.
 0.28728295 0.19068559], Fit: -45.514622000951704, nEval: 60
[2] Best: [0.00582515 0.09248096 0.11642007 0.33060967 0.2519999  0.
 0.16198436 0.04067989], Fit: -46.057916190894616, nEval: 80
[3] Best: [0.         0.11207504 0.         0.20168837 0.38968958 0.
 0.15749996 0.13904704], Fit: -50.34041210388189, nEval: 100
[4] Best: [0.         0.         0.         0.28892395 0.32255979 0.00590613
 0.20010667 0.18250347], Fit: -50.4113035638918, nEval: 120
[5] Best: [0.00688202 0.12346164 0.         0.45434611 0.25830636 0.
 0.15700386 0.        ], Fit: -51.36829851275775, nEval: 140
[6] Best: [0.         0.13543064 0.09092888 0.04363733 0.38211477 0.
 0.34788837 0.        ], Fit: -57.562049209390416, nEval: 160
[7] Best: [0.         0.13543064 0.09092888 0.04363733 0.38211477 0.
 0.34